In [32]:
from experiments.utilities import *

In [34]:
data_path='/Users/maoss2/Documents/Doctorat/DATASET_CANCERS/BRCA'
data_repository = '/Users/maoss2/PycharmProjects/BRCA_experiments_and_paper/datasets/datasets_repository'
dico = defaultdict(dict)
os.chdir(f'{data_path}')

In [4]:
for fichier in glob('*.tsv'):
    d = pd.read_csv(fichier, sep='\t')
    d = d.loc[d['combined_score'] >= 0.5]
    for noeud in d['#node1'].values:
        if noeud not in dico.keys():
            dico[noeud] = list(d.loc[d['#node1'] == noeud]['node2'].values)
        else:
            dico[noeud].extend(list(d.loc[d['#node1'] == noeud]['node2'].values))
            


In [7]:
for k, v in dico.items():
    dico[k] = list(np.unique(np.asarray(v)))

In [22]:
string_pathways = [list(v) + [k] for k, v in dico.items()]
string_pathways.append(['miRNA']) # biogrid_pathways[20170]
string_pathways.append(['clinical View']) # biogrid_pathways[20171]
string_pathways.append(['unknown'])

In [28]:
with open(f'{data_repository}/string_pathways_list.pck', 'wb') as f:
    pickle.dump(string_pathways, f)

In [35]:
data_tn_new_label_unbalanced_cpg_rna_rna_iso_mirna = f"{data_repository}/triple_neg_new_labels_unbalanced_cpg_rna_rna_iso_mirna.h5"
_, _, features_names, _ = load_data(data=data_tn_new_label_unbalanced_cpg_rna_rna_iso_mirna, 
                                    return_views='all')

In [43]:
features_names = list(features_names)
dico_results = {feature: [] for feature in features_names}

In [44]:
for feature in features_names:
    if feature.find('_') != -1:
        # I went step by step for the comprehension but remember the gene is always at the end of the feature so
        # use the [-1] access
        split_results = feature.split('_')
        gene_cible = split_results[-1]
        if gene_cible.find(';'): # Cas ou on a un feature lié à 2 ou plus genes (surtout pour les cpg)
            gene_cibles = gene_cible.split(';')
            for gene in gene_cibles: # Recupere chaque gene et on remplit le dico
                for idx, pathway_in_biogrid in enumerate(string_pathways):
                    if gene in pathway_in_biogrid:
                        dico_results[feature].append('G_{}'.format(idx))
        else: # Different du premier if du coup le feature est link à un seul gene et on remplit le dictionnaire de facon adéquate
            for idx, pathway_in_biogrid in enumerate(string_pathways):
                if gene_cible in pathway_in_biogrid:
                    dico_results[feature].append('G_{}'.format(idx))
    elif feature.find('|') != -1: # Here the gene is the 1st element always since it's directly the RNA view only
        split_results = feature.split('|')
        gene_cible = split_results[0]
        for idx, pathway_in_biogrid in enumerate(string_pathways):
            if gene_cible in pathway_in_biogrid:
                dico_results[feature].append('G_{}'.format(idx))
    elif feature.startswith('hsa'):  # MiRNA View: faire le traitement directement
        dico_results[feature].append(string_pathways[-3])
    else:
        dico_results[feature].append(string_pathways[-2])
for cle, valeur in dico_results.items():
    if valeur == []:
        dico_results[cle].append(string_pathways[-1])


In [45]:
dico_results

{'EFNA1|1942': ['G_2621',
  'G_2623',
  'G_2630',
  'G_2663',
  'G_2673',
  'G_2731',
  'G_2782'],
 'hsa-mir-1307': ['miRNA'],
 'uc004bvu.3_SET': ['G_338'],
 'uc002uqk.2_COL5A2': ['G_1444',
  'G_1525',
  'G_1528',
  'G_1563',
  'G_1595',
  'G_1602',
  'G_1615',
  'G_1622',
  'G_1641',
  'G_1644'],
 'EWSR1|2130': ['G_2930', 'G_2945'],
 'uc001fbw.1_S100A6': ['G_85', 'G_336', 'G_432', 'G_443'],
 'GUSB|2990': ['G_9286', 'G_9399'],
 'uc002cqs.2_PDPK1': ['G_8532', 'G_8616', 'G_8618', 'G_8651', 'G_8684'],
 'cg04599297_GAD2': ['G_6881',
  'G_6900',
  'G_6904',
  'G_6912',
  'G_6979',
  'G_6983',
  'G_6984',
  'G_6987',
  'G_6991',
  'G_7007',
  'G_7105',
  'G_7187',
  'G_7222'],
 'FADS2|9415': ['G_2734', 'G_2736', 'G_3079', 'G_3089'],
 'cg25720804_TLX3': ['G_4714'],
 'uc002kal.1_ACTG1': ['G_3710',
  'G_3720',
  'G_3721',
  'G_3862',
  'G_3870',
  'G_3937',
  'G_4079',
  'G_4120',
  'G_4121'],
 'cg25691167_FERD3L': ['unknown'],
 'FTL|2512': ['G_6880', 'G_6967'],
 'cg20162599_FERMT3': ['G_6891']

In [51]:
with open(f'{data_repository}/pathways_string_groups.pck', 'wb') as f:
    pickle.dump(dico_results, f)

In [52]:
temp_dict = {f'G_{idx}': el for idx, el in enumerate(string_pathways)}

In [54]:
with open(f'{data_repository}/string_pathways_dict.pck', 'wb') as f:
    pickle.dump(temp_dict, f)

'/Users/maoss2/Documents/Doctorat/DATASET_CANCERS/BRCA'